In [1]:
#Skal bruges hvis man er på erda ellers kan man bare udkommentere den

#pip install keras-tcn    #Skal bruges hver gang man genstarter erda

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tcn import TCN
from tensorflow import keras
import tensorflow as tf
from sklearn import metrics
from HomemadeFuncs import ROC_plot
import sqlite3
import time
from Dataloader import *

## Parameters

In [3]:
N_muons = 80000
N_epochs = 10              #Prøvede 100 epochs men ændrede ikke noget
N_filters = 64
Kernel_size = 2
lr = 0.001
N_features = 5
batch_size = 1000           #200+ giver OOM (Out of memory på min pc)
input_shape = (batch_size, None, 5)
output_dim = 2

params  = {'optimizer': tf.keras.optimizers.Adam(learning_rate=lr),
           'loss'     : tf.keras.losses.BinaryCrossentropy(),
           'metrics'  : [tf.keras.metrics.AUC() , tf.keras.metrics.BinaryAccuracy()]}

In [4]:
db_file = './rasmus_classification_muon_3neutrino_3mio.db'
all_events = pd.read_csv('mine_events.csv').values
events = all_events[0:N_muons]
events = np.reshape(events, (N_muons,))

In [5]:
# Alle events skal paddes til samme længde, i.e. længden af det længste event
max_length = 1
for event in events:
    features, _ = PullFromDB(event, db_file)
#    print(features.shape[0])
    if features.shape[0] > max_length:
        max_length = features.shape[0]
max_length

3315

# Model Architecture

In [6]:
def tcn_model_26():
    i = tf.keras.Input(batch_shape = (None, max_length,5))
    o = TCN(
        nb_filters = N_filters,
        kernel_size = Kernel_size,
        #nb_stacks = 1,
        #dilations=[1, 2, 4, 8, 16, 32],
        #padding="causal",
        #use_skip_connections = False,
        dropout_rate = 0.2,                                     #test/forslag fra Johann
        #return_sequences = False,
        activation = tf.keras.activations.tanh,#tf.keras.layers.LeakyReLU()#
        #kernel_initializer="he_normal",
        #use_layer_norm=False,
        #use_batch_norm = False,
     )(i)
    o = tf.keras.layers.Dense(N_filters, activation = "relu")(o) #test/forslag fra Johann
    o = tf.keras.layers.Dropout(0.2)(o)                          #test/forslag fra Johann
    o = tf.keras.layers.Dense(N_filters, activation = "relu")(o) #test/forslag fra Johann
    
    o = tf.keras.layers.Dense(output_dim, activation = 'sigmoid')(o)
    model = tf.keras.models.Model(inputs=[i], outputs=[o])
    return model

In [7]:
model = tcn_model_26()
model.compile(optimizer=params['optimizer'], loss=params['loss'], metrics= params['metrics'])

## Training

In [8]:
def Train():
    with tf.device('/GPU:0'):     #GPU aktivering
        a,b = 0, batch_size
        
        for i in range(int(N_muons/batch_size)):
            input_feature_list = []
            input_targets_list = []
            
        #Trækker en batch af events ud  
            for event in events[a:b]:
                features, truth = PullFromDB(event, db_file, E_threshold=0)
                input_feats = TransformFeaturesToInput(features)
                input_feature_list.append(input_feats)
                input_targs = TransformTruthToInput(truth)
                input_targets_list.append(input_targs)
                
        #Padder events, så hver batch har samme dimensioner
            padded_input = []
    
            for feature in input_feature_list:
                    temp = np.zeros([1, max_length, 5])
                    temp[:, :feature.shape[1], :] = feature
                    padded_input.append(temp)        
                    padded_features = np.concatenate(padded_input)
                    padded_target = np.concatenate(input_targets_list)
    
        #Træner på "batch_size" events af gangen
            model.fit(padded_features, padded_target, epochs = N_epochs, verbose=0, batch_size = batch_size)
            
        # Print progress
            if i % 10 ==0:
                print((1+i)*batch_size,'of',N_muons,'||', np.round(100*(batch_size*(i+1)/(N_muons)),2), '%')

            a += batch_size
            b += batch_size
    print('Done')

In [9]:
with tf.device('/GPU:0'):     #GPU aktivering
    start = time.time()
    Train()
    end = time.time()
    T = np.round(end-start, 2)

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[1000,64,1,3316] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradient_tape/functional_1/tcn/residual_block_0/conv1D_1/conv1d/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_29]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[1000,64,1,3316] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradient_tape/functional_1/tcn/residual_block_0/conv1D_1/conv1d/Conv2DBackpropFilter-0-TransposeNHWCToN

In [ ]:
print('Training time for ' + f'{int(N_muons/1000)}' + 'k events ' + (f'({N_epochs}' + ' epochs): '))
if 0<T<180:
    print(T, 's')
elif 180 < T < 3600:
    print(T/60, 'm')
else:
    print(T/3600, 'hrs')

In [ ]:
model.summary()

## Save/Load Weights

In [ ]:
model_name = str(int(N_muons/1000)) + 'km-' + str(N_epochs) + 'e'
model_name
date = time.strftime("%d-%m")
date
weightsname = model_name + '_' + date + '.h5'
weightsname

In [ ]:
#model.save_weights(weightsname)

In [ ]:
#weights_file = './filename.h5'
#model.load_weights(weights_file)

# Predictions

In [ ]:
N_preds = 100

def Predict(use_training_data = True):
    with tf.device('/GPU:0'):        #GPU aktivering
        if use_training_data == True:
            Events = events[:N_preds]
        else:
            c = int(N_muons + N_preds)
            NT_events = np.reshape(all_events[N_muons:c], (N_preds,))
            Events = NT_events
            
        preds = []
        targs = []
        
        input_feature_list = []
        
        for event in Events:
            features, truth = PullFromDB(event, db_file, E_threshold=0)
            input_feats = TransformFeaturesToInput(features)
            input_feature_list.append(input_feats)
            targs.append(truth.values[0][0])
            
            padded_input = []
        for feature in input_feature_list:
            temp = np.zeros([1, max_length, 5])
            temp[:, :feature.shape[1], :] = feature
            padded_input.append(temp)        
            padded_features = np.concatenate(padded_input)
            
        preds.append(model.predict(padded_features))
            #preds = model.predict(padded_features)
        
        return np.array(preds), targs

In [ ]:
#with tf.device('/GPU:0'):
start = time.time()
preds, targs = Predict()
end = time.time()
Tt = np.round(end-start,2)
print('Time:',Tt, 's')

In [ ]:
preds.shape

# Results

In [ ]:
predsarray = np.reshape(np.array(preds), (N_preds,2))

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.hist(predsarray[:, 0], bins = 50, alpha=0.75, label='Probability of through')
ax.hist(predsarray[:, 1], bins = 50, alpha=0.75, label='Probability of stopped')
ax.legend()
plt.show()

In [ ]:
ROC_plot(predsarray, targs)

In [ ]:
unique_preds, counts_preds = np.unique(predsarray[:,0], return_counts=True)
print('Unique through predictions: ', np.round(unique_preds, 3))
print('Counts through predictions: ', counts_preds)

In [ ]:
unique_targs, counts_targs = np.unique(predsarray[:,1], return_counts=True)
print('Unique stopped predictions: ', np.round(unique_targs, 3))
print('Counts stopped predictions: ', counts_targs)

In [ ]:
guesses = np.zeros((N_preds,1))
for i in range(N_preds):
    guesses[i] = np.argmax(preds[0][i])

unique_preds, counts_preds = np.unique(guesses, return_counts=True)
print('Unique predictions: ', unique_preds)
print('Counts predictions: ',counts_preds)

In [ ]:
unique_targs, counts_targs = np.unique(targs, return_counts=True)
print('Unique targets: ', unique_targs)
print('Counts targets: ',counts_targs)